# HDL playground

This HDL playground is based on the `hackfin/myhdl_testing:jupyosys` Docker container. The underlying toolchain is the [yosys](https://github.com/YosysHQ/yosys/) synthesis toolchain.

It is meant to collect examples and test scenarios for various Hardware Definition Languages.

The top level default command language is Python, with frequent usage of MyHDL for test benches, co-simulation or HDL modules to be emitted as Verilog and/or VHDL implementation for functional comparison or verification.

Supported HDLs in order of maturity with respect to yosys:

* [Verilog](#Verilog-support)
* [VHDL](#VHDL-support) (simulation and synthesis using GHDL)
* [MyHDL](#MyHDL-support) (through jupyosys)

Automated testing of notebook is integrated [below](#Automated-notebook-tests).

## Examples

### Third party projects

* [1pCPU](third_party.ipynb#1pCPU) - a minimal 8 bit CPU on one page

## Introduction

### Simulation issues

There is an issue with specific files that are required to simulate hardware primitives:

* They are vendor-proprietary
* Possibly under NDA or encrypted

If you want to co-simulate against a vendor entity, like from an ECP5 FPGA, the current procedure for this architecture is as follows:

1. Obtain the `*.v` files needed for simulation from your installed toolchain
+  Copy those files into the directory specified by the `ECP5_LIB` variable in [config.py](#File-edit-links),
see also Upload link below.

An 'enhanced' `cells_sim.v` file is added for example that includes some vendor simulation files ([cells_sim.v](#File-edit-links)). You need to uncomment the lines on the bottom after installation of the necessary `*.v` files.

**Note**: The vendor simulation files should not be processed by yosys, as they contain non-synthesizable Verilog
statements. From the yosys side, these entities appear as blackbox cells, later on, the icarus verilog simulator
includes them for proper co-simulation.

### File edit links

We need to generate the edit links to run both on a local container and the binderhub service (the code to generate them is hidden by default)

In [1]:
from tests.pgaux import hide_toggle
hide_toggle(for_next = True)

In [8]:
from tests.pgaux import generate, Markdown

d = ("Upload vendor files (use the `Upload` button)", 'tests/lib/techlibs/ecp5')
f = [ ("Check local configuration", 'tests/config.py'), ("Uncomment include statements", 'tests/lib/techlibs/ecp5/cells_sim.v')]

s = generate(d, f)
Markdown("### Steps to **edit** files:\n\n" + s)

### Steps to **edit** files:

  * Upload vendor files (use the `Upload` button): [tests/lib/techlibs/ecp5](None/tree/work/tests/lib/techlibs/ecp5)

  * Check local configuration: <a href="/edit/work/tests/config.py" target="_blank">tests/config.py</a>
  * Uncomment include statements: <a href="/edit/work/tests/lib/techlibs/ecp5/cells_sim.v" target="_blank">tests/lib/techlibs/ecp5/cells_sim.v</a>

## Verilog support

Verilog support is built-in into this container:

* Simulation: [iverilog](http://iverilog.icarus.com/) simulator
* Synthesis and verification: [yosys](www.clifford.at/yosys/), [nextpnr](https://github.com/YosysHQ/nextpnr)

## VHDL support

VHDL support for simulation and synthesis (preliminary) is integrated via [GHDL](https://github.com/ghdl/ghdl/) and the [ghdl-yosys-plugin](https://github.com/ghdl/ghdl-yosys-plugin).

The GHDL yosys plugin is automatically compiled during the build process of this container.

Proceed to [GHDL overview](ghdl.ipynb) containing:
* Build and simulate blinky
* Synthesis and verification
* Download to ECP5 FPGA target

## MyHDL support

Direct (experimental) Synthesis through MyHDL is featured by the [jupyosys](https://section5.ch/index.php/2020/03/20/myhdl-and-pyosys-direct-synthesis-using-python/) project, see separate Binder:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/hackfin/myhdl.git/jupyosys?filepath=src%2Fmyhdl%2Fmyhdl-yosys%2Fexample%2Fipynb%2Findex.ipynb)


# Complex SoC design auxiliaries

Large projects may require some framework to keep the maintenance overhead low, while having dependencies resolved during continous integration builds. This has lead to a few developments with respects to abstract property aware description languages.

* [netpp/devdesc](https://section5.ch/netpp/): A device description XML dialect and boundary agnostic RPC protocol (network property procotol)
* [XML device description](xml_devdesc.ipynb) examples:
    * gensoc: A tool to generate code from a devdesc XML device description
    * xsltproc: Creating SVG graphics from register maps using XSL (XML style sheets)
* [MaSoCist](https://github.com/hackfin/masocist)
   * Application scenarios for devdesc, gensoc
   * A build system to configure, build, test, document and synthesize a family of SoCs

# Automated notebook tests

Automated notebook tests are run with the code below to verify that:

* There is no exception occuring during execution
* The output matches the previous notebook output

It may be that a notebook shows a `FAILED` state and passes upon regeneration and the next autotest run. For strict comparison, remove `--nb-force-regen` below.

In [3]:
! pip3 install pytest-notebook >/dev/null

Install the magic extension:

In [2]:
%load_ext pytest_notebook.ipy_magic

We have to manually configure the pytest executable:

In [5]:
import pytest_notebook
pytest_notebook.ipy_magic.EXEC_NAME = "py.test-3"

### Running the test suite

In [7]:
%%pytest -v --color=yes --disable-warnings --nb-exec-timeout 50 --nb-force-regen

---
[pytest]
nb_test_files = True
nb_diff_ignore = 
    /cells/*/outputs/*/data/image/svg+xml
nb_diff_replace =
    /cells/*/outputs/*/data/text .*graphviz\.files\.Source.* "GRAPH"
    /cells/*/outputs/*/data/text .*graphviz\.dot\.Digraph.* "GRAPH"
    /cells/*/outputs/*/text .*-rw\-r\-\-r\-\-.* "FILE"
    /cells/*/outputs/*/text .*End.of.script.* "YOSYS_TAG"

---

import sys
import importlib_resources

sys.path.append("/home/pyosys/work")
sys.path.append("/home/pyosys/work/examples")

import pytest
import tests
import tempfile

        
VERIFIED  = [ "ghdl_module_tests.ipynb", "ghdlsim_blink.ipynb", "memory_wrapped_init.ipynb" ]

KNOWN_TO_FAIL = [ "memory.ipynb" ]

@pytest.mark.parametrize("uut", VERIFIED)
def test_generic(nb_regression, uut):
    with importlib_resources.path(tests, uut) as path:
        nb_regression.check(str(path))                

@pytest.mark.xfail 
@pytest.mark.parametrize("uut", KNOWN_TO_FAIL)
def test_fixme(nb_regression, uut):
    with importlib_resources.path(tests, uut) as path:
        nb_regression.check(str(path))                              

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-3.10.1, py-1.7.0, pluggy-0.8.0 -- /usr/bin/python3
cachedir: .pytest_cache
NB force regen: True
rootdir: /tmp/tmpttl05q9j, inifile: pytest.ini
plugins: notebook-0.6.0
collecting ... collected 4 items

test_ipycell.py::test_generic[ghdl_module_tests.ipynb] PASSED            [ 25%]
test_ipycell.py::test_generic[ghdlsim_blink.ipynb] PASSED                [ 50%]
test_ipycell.py::test_generic[memory_wrapped_init.ipynb] PASSED          [ 75%]
test_ipycell.py::test_fixme[memory.ipynb] xfail                          [100%]

===================== 3 passed, 1 xfailed in 51.00 seconds =====================
